##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/pytorch_gemma"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/pytorch_gemma.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemma/docs/pytorch_gemma.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Gemma in PyTorch

This is a quick demo of running Gemma inference in PyTorch.
For more details, please check out the Github repo of the official PyTorch implementation [here](https://github.com/google/gemma_pytorch).

**Note that**:
 * The free Colab CPU Python runtime and T4 GPU Python runtime are sufficient for running the Gemma 2B models and 7B int8 quantized models.
 * For advanced use cases for other GPUs or TPU, please refer to [README.md](https://github.com/google/gemma_pytorch/blob/main/README.md) in the official repo.

## Kaggle access

To login to Kaggle, you can either store your `kaggle.json` credentials file at
`~/.kaggle/kaggle.json` or run the following in a Colab environment. See the
[`kagglehub` package documentation](https://github.com/Kaggle/kagglehub#authenticate)
for more details.

In [1]:
import kagglehub

kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


## Install dependencies

In [2]:
!pip install -q -U torch immutabledict sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.


## Download model weights

In [3]:
# Choose variant and machine type
VARIANT = '2b-it' #@param ['2b', '2b-it', '7b', '7b-it', '7b-quant', '7b-it-quant']
MACHINE_TYPE = 'cuda' #@param ['cuda', 'cpu']

In [4]:
import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

100%|██████████| 3.75G/3.75G [03:27<00:00, 19.4MB/s]
Extracting model files...


## Download the model implementation

In [5]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 148 (delta 46), reused 38 (delta 23), pack-reused 68
Receiving objects: 100% (148/148), 2.16 MiB | 6.88 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [6]:
import sys

sys.path.append('gemma_pytorch')

In [7]:
from gemma_pytorch.gemma.config import get_config_for_7b, get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

## Setup the model

In [8]:
import torch

# Set up model config.
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

## Run inference

Below are examples for generating in chat mode and generating with multiple
requests.

The instruction-tuned Gemma models were trained with a specific formatter that
annotates instruction tuning examples with extra information, both during
training and inference. The annotations (1) indicate roles in a conversation,
and (2) delineate turns in a conversation. Below we show a sample code snippet
for formatting the model prompt using the user and model chat templates in a
multi-turn conversation. The relevant tokens are:

- `user`: user turn
- `model`: model turn
- `<start_of_turn>`: beginning of dialogue turn
- `<end_of_turn>`: end of dialogue turn

Read about the Gemma formatting for instruction tuning and system instructions
[here](https://ai.google.dev/gemma/docs/formatting).

In [15]:
# 헬스 트레이너 페르소나 설정
persona = "나는 친절하고 격려적인 헬스 트레이너 제이입니다. 사용자에게 맞춤형 운동 계획과 식단 조언을 제공하고, 목표 달성을 돕습니다."

# Chat templates
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

# 샘플 대화 프롬프트
prompt = (
    USER_CHAT_TEMPLATE.format(prompt='체중 감량을 위해 어떤 운동을 시작해야 할까요?')
    + MODEL_CHAT_TEMPLATE.format(prompt='처음이라면 걷기나 가벼운 조깅부터 시작해보세요. 점차 강도를 높일 수 있습니다.')
    + USER_CHAT_TEMPLATE.format(prompt='식단은 어떻게 해야 할까요?')
    + '<start_of_turn>model\n'
)
print('Chat prompt:\n', prompt)

model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=400,
)


Chat prompt:
 <start_of_turn>user
체중 감량을 위해 어떤 운동을 시작해야 할까요?<end_of_turn>
<start_of_turn>model
처음이라면 걷기나 가벼운 조깅부터 시작해보세요. 점차 강도를 높일 수 있습니다.<end_of_turn>
<start_of_turn>user
식단은 어떻게 해야 할까요?<end_of_turn>
<start_of_turn>model



'체중 감량을 위해 좋은 식단은 기본적으로 과일, 채소, 견과류와 단백질이 포함된 식단을 선택하는 것이 좋습니다.'

In [16]:
# Generate sample
model.generate(
    '체중 감량 어떻게 해?',
    device=device,
    output_len=1000,
)

'\n\n체중 감량은 식이를 줄이고 운동을 시작하는 과정입니다. 체중 감량에는 여러 방법이 있지만, 세 가지 주요 방법을 이름해 드리겠습니다.\n\n1. **식단과 운동 조합**\n2. **주의 식이**\n3. **체중 감량 지표**\n\n**식단과 운동 조합**\n\n식단과 운동을 조합하는 것은 가장 유력한 체중 감량 방법입니다. 이 방법은 식단과 운동의 각 부분이 효과적으로 작용함을 통해 체중 감량을 촉진합니다. 식단은 단백질과 탄수화물의 비율을 높이고, 운동은 지속적으로 근육을 생성하는 데 도움이 되는 칼로리를 절개하는 데 사용됩니다.\n\n**주의 식이**\n\n주의 식이는 과일, 채소 및 단백질 함량이 높은 식단을 제시하는 것을 의미합니다. 과일은 채소의 섬유성과 단백질 함량에 높습니다. 단백질은 올리브 오일, 콩, nuts, and seeds에 풍요롭습니다.\n\n**체중 감량 지표**\n\n체중 감량 지표는 식이와 운동의 효과를 모니터링하기 위해 사용하는 지표입니다. 체중 감량 지표는 체중, 비만도, 혈액 순환, 혈액 청소, 지방 수준 및 콜레스테롤 수준을 포함합니다. 체중 감량 지표는 목표 체중에 도달하기 위해 사용할 수 있는 유용한 지표입니다.'

In [11]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.2 MB/

In [12]:
!pip install --upgrade gradio

In [18]:
import gradio as gr

# 헬스 트레이너 페르소나 설정
persona = "나는 친절하고 격려적인 헬스 트레이너 제이입니다. 사용자에게 맞춤형 운동 계획과 식단 조언을 제공하고, 목표 달성을 돕습니다."
conversation_history = []

def gemma_chat(user_input):
    global conversation_history

    prompt = f"{persona}\n\n{humanized_conversation_history(conversation_history)}\n<start_of_turn>user\n{user_input}<end_of_turn>\n<start_of_turn>model\n"
    model_output = model.generate(prompt, device=device, output_len=1000)

    conversation_history.append(f"<start_of_turn>user\n{user_input}<end_of_turn>")
    conversation_history.append(f"<start_of_turn>model\n{model_output}<end_of_turn>")

    return model_output

def humanized_conversation_history(conversation):
    humanized = ""
    for turn in conversation:
        if "<start_of_turn>user" in turn:
            humanized += "User: " + turn.split(">")[-1] + "\n"
        elif "<start_of_turn>model" in turn:
            humanized += "Gemma: " + turn.split(">")[-1] + "\n"
    return humanized

iface = gr.Interface(gemma_chat,
                     inputs=gr.Textbox(lines=2, placeholder="운동이나 식단에 대한 질문을 입력하세요..."),
                     outputs="text",
                     title="헬스 트레이너 제이")

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bcdd2d6a33989d2a98.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Learn more

Now that you have learned how to use Gemma in Pytorch, you can explore the many
other things that Gemma can do in [ai.google.dev/gemma](https://ai.google.dev/gemma).
See also these other related resources:

- [Gemma model card](https://ai.google.dev/gemma/docs/model_card)
- [Gemma C++ Tutorial](https://ai.google.dev/gemma/docs/gemma_cpp)
- [Gemma formatting and system instructions](https://ai.google.dev/gemma/docs/formatting)